In [1]:
import numpy as np
import pandas as pd

def simulate_hhw_paths(
    S0: float,
    v0: float,
    r0: float,
    kappa_v: float,
    theta_v: float,
    sigma_v: float,
    kappa_r: float,
    theta_r: float,
    sigma_r: float,
    rho_sv: float,
    rho_sr: float,
    rho_vr: float,
    T: float,
    N: int,
    M: int,
    seed: int = None
) -> dict:
    """
    Simulate Monte Carlo paths under the Heston-Hull-White model and return a dict of DataFrames.

    Each key in the returned dict corresponds to a path index, and the DataFrame contains columns:
      - 'S': simulated asset prices
      - 'v': simulated variances
      - 'r': simulated short rates

    Returns:
    -------
    dfs : dict[int, pd.DataFrame]
        Dictionary mapping path index to a DataFrame of shape (M+1, 3).
    """
    dt = T / M

    # Build correlation matrix and its Cholesky factor
    corr = np.array([
        [1.0, rho_sv, rho_sr],
        [rho_sv, 1.0, rho_vr],
        [rho_sr, rho_vr, 1.0]
    ])
    L = np.linalg.cholesky(corr)

    # Arrays to store paths
    S = np.zeros((M + 1, N))
    v = np.zeros((M + 1, N))
    r = np.zeros((M + 1, N))
    S[0, :] = S0
    v[0, :] = v0
    r[0, :] = r0

    rng = np.random.default_rng(seed)
    for t in range(1, M + 1):
        Z = rng.standard_normal((3, N))
        dW = (L @ Z) * np.sqrt(dt)
        dW1, dW2, dW3 = dW

        # Full-truncation Euler for variance
        v_prev = np.maximum(v[t-1], 0)
        v[t] = (
            v[t-1]
            + kappa_v * (theta_v - v_prev) * dt
            + sigma_v * np.sqrt(v_prev) * dW2
        )
        v[t] = np.maximum(v[t], 0)

        # Hull-White short rate
        r[t] = (
            r[t-1]
            + kappa_r * (theta_r - r[t-1]) * dt
            + sigma_r * dW3
        )

        # Asset price log-Euler
        S[t] = (
            S[t-1]
            * np.exp((r[t-1] - 0.5 * v_prev) * dt + np.sqrt(v_prev) * dW1)
        )

    # Build DataFrame dictionary
    dfs = {
        i: pd.DataFrame({'S': S[:, i], 'v': v[:, i], 'r': r[:, i]})
        for i in range(N)
    }
    return dfs


def compute_risk_control_index(
    df: pd.DataFrame,
    lambda_s: float = 0.94,
    lambda_l: float = 0.97,
    target_volatility: float = 0.1,
    max_k: float = 1.5,
    lag: int = 2,
    initial_value: float = 100.0,
    initial_var: float = 0.04/252,
    interest_rate_basis: float = 0.02963
) -> pd.DataFrame:
    """
    Attach a risk control index to a single-path DataFrame.

    Adds columns:
      - 'risky_asset_returns'
      - 'variance_s', 'variance_l'
      - 'realized_volatility'
      - 'K', 'interest_rate'
      - 'risk_control_index_value'
    """
    df = df.copy()
    df['risky_asset_returns'] = df['S'].pct_change().fillna(0)

    var_s, var_l = initial_var, initial_var
    vs, vl = [], []
    for ret2 in df['risky_asset_returns']**2:
        vs = (vs + [(1-lambda_s)*ret2 + lambda_s*var_s]) if False else vs
        vl = (vl + [(1-lambda_l)*ret2 + lambda_l*var_l]) if False else vl
        # incremental update
        var_s = (1-lambda_s)*ret2 + lambda_s*var_s
        var_l = (1-lambda_l)*ret2 + lambda_l*var_l
        vs.append(var_s)
        vl.append(var_l)
    df['variance_s'], df['variance_l'] = vs, vl

    # annualize and take max
    vol_s = np.sqrt(df['variance_s'] * 252)
    vol_l = np.sqrt(df['variance_l'] * 252)
    df['realized_volatility'] = pd.concat([vol_s, vol_l], axis=1).max(axis=1)
    df['realized_volatility'] = df['realized_volatility'].shift(lag).bfill()

    # leverage ratio
    df['K'] = (target_volatility / df['realized_volatility']).clip(upper=max_k)

    # funding cost
    df['interest_rate'] = df['r'].add(interest_rate_basis).shift(1)
    df['interest_rate'] /= 252

    # index returns and values
    df['rc_index_ret'] = df['K'] * (df['risky_asset_returns'] - df['interest_rate'])
    df['rc_index_value'] = (1 + df['rc_index_ret']).cumprod() * initial_value

    return df


if __name__ == "__main__":
    # Simulation settings
    S0, v0, r0 = 100.0, 0.04, 0.03
    sim_params = dict(
        kappa_v=2.0, theta_v=0.04, sigma_v=0.3,
        kappa_r=0.1, theta_r=0.03, sigma_r=0.01,
        rho_sv=-0.7, rho_sr=0.2, rho_vr=0.1,
        T=1.0, N=1000, M=252, seed=42
    )
    dfs = simulate_hhw_paths(S0, v0, r0, **sim_params)
    
    # Compute risk-control for path 0
    rc_df = compute_risk_control_index(
        dfs[0],
        lambda_s=0.94, lambda_l=0.97, target_volatility=0.1,
        max_k=1.5, lag=2,
        initial_value=100.0, initial_var=0.04/252,
        interest_rate_basis=0.02963
    )
    print(rc_df[['S', 'risky_asset_returns', 'K', 'rc_index_value']].head())


            S  risky_asset_returns         K  rc_index_value
0  100.000000             0.000000  0.507673             NaN
1  100.388629             0.003886  0.507673      100.185284
2  101.965507             0.015708  0.507673      100.972213
3  102.246659             0.002757  0.514684      101.103247
4  102.371546             0.001221  0.509760      101.154055


In [2]:
rc_df

,S,v,r,risky_asset_returns,variance_s,variance_l,realized_volatility,K,interest_rate,rc_index_ret,rc_index_value
0,100.000000,0.040000,0.030000,0.000000,0.000149,0.000154,0.196977,0.507673,NaN,NaN,NaN
1,100.388629,0.039034,0.029764,0.003886,0.000141,0.000150,0.196977,0.507673,0.000237,0.001853,100.185284
2,101.965507,0.036452,0.029825,0.015708,0.000147,0.000153,0.196977,0.507673,0.000236,0.007855,100.972213
3,102.246659,0.032148,0.029738,0.002757,0.000139,0.000148,0.194294,0.514684,0.000236,0.001298,101.103247
4,102.371546,0.032392,0.029837,0.001221,0.000131,0.000144,0.196171,0.509760,0.000236,0.000503,101.154055
...,...,...,...,...,...,...,...,...,...,...,...
248,94.920963,0.055783,0.024371,-0.004058,0.000191,0.000191,0.231337,0.432269,0.000212,-0.001846,95.547242
249,95.803055,0.052042,0.024786,0.009293,0.000184,0.000188,0.225434,0.443588,0.000214,0.004027,95.932026
250,97.870091,0.048428,0.025167,0.021576,0.000201,0.000196,0.219436,0.455713,0.000216,0.009734,96.865830
251,98.686650,0.040233,0.024347,0.008343,0.000193,0.000192,0.217625,0.459506,0.000217,0.003734,97.227515
